In [113]:
import openrouteservice as ors
from openrouteservice import *
import ipyleaflet as leaf
from ipyleaflet import *
import ipywidgets as widgets
from ipywidgets import *
import folium as f
from folium import *



#_____OPENROUTESERVICE: API KEY & CALL TO CLIENT_____________________________________________________________________
ors_key = '5b3ce3597851110001cf624870ced9cc33b74f5a9c86a23e06e5143e'
client = ors.Client(key=ors_key)

#_____IPYLEAFLET: INITIALIZE MAP _____________________________________________________________________
center = [36.1699, -115.1398]
bragg_map = leaf.Map(basemap=leaf.basemap_to_tiles(leaf.basemaps.OpenStreetMap.Mapnik), center=center, zoom=6, max_zoom=18, min_zoom=3, scroll_wheel_zoom=True, layout=widgets.Layout(height='680px', width='auto'))
layer_control = leaf.LayersControl(position='topright')
bragg_map.add_control(layer_control)
bragg_map.add_control(leaf.FullScreenControl(position='topleft'))


#_____FOLIUM: ISOCHRONE STYLE FUNCTIONS______________________________________________________________________
def style_bragg(feature):
    return {'color':'#000000',  'weight': '1', 'fillColor': '#fcb615' , 'fillOpacity': '0.08'}
def style_competitor(feature):
    return {'color': '#CC0000', 'weight': '1',  'fillColor': '#CC0000', 'fillOpacity': '0.08'}
def style_jobsite(feature):
    return { 'color': '#006600', 'weight': '1',  'fillColor': '#006600', 'fillOpacity': '0.08'}
def style_route(feature):
    return {'color':'#FD1C03', 'weight':'2'}


#_____IPYWIDGETS: HBOX/VBOX/BUTTON LAYOUT & STYLING______________________________________________________________________
hbox_lay = widgets.Layout(justify_content='space-between')
button_lay_add = widgets.Layout(height='30px', flex='1 1 auto', width='auto', border='1.5px outset #e2a312')
button_lay_remove = widgets.Layout(height='30px', flex='1 1 auto', width='auto', border='1.5px outset #646464')
coord_lay = widgets.Layout(width='97px')
vbox_lay = widgets.Layout(padding='auto auto 0px auto', margin='2px 2px 2px 2px')
slider_lay = widgets.Layout(width='207px')
other_lay = widgets.Layout(width='264px')
buttons_lay = widgets.Layout(margin='5px auto 0px auto', display='flex', flex_flow='row', align_items='stretch', width='100%')
addb_style = widgets.ButtonStyle(button_color = '#fcb615', font_weight = '650')
removeb_style = widgets.ButtonStyle(button_color='#6d6d6d', font_weight = '650')

#_____OPENROUTESERVICE: API CALLS___________________________________________________________________________________

#___ISOCHRONE___________________________________________________________________________________
def compute_poly(latitude, longitude, unit_type, total_range, intervals, smooth):
    global client                                        # latitude/longitude(type: float)
    coordinates = [[longitude, latitude]]                # unit_type(type: str)- calculate polygons using 'distance' or 'time'
    if unit_type == 'time':                              # total_range(type: int)- total size of isochrone in units 
        total_range *= 60
        intervals *= 60                                                    # intervals(type: int)- units to segment isochrone by 
    if (intervals == 0 or intervals == total_range):
        poly = client.isochrones(locations = coordinates,    
                            profile = 'driving-hgv',
                            range_type = unit_type,
                            range = [total_range], 
                            smoothing = smooth,
                            units = 'mi')
    else:
        poly = client.isochrones(locations = coordinates,    
                            profile = 'driving-hgv',
                            range_type = unit_type,
                            range = [total_range],
                            interval = intervals, 
                            smoothing = smooth,
                            units = 'mi')
    return poly


#_____ROUTE______________________________________________________________________________________
def find_route(coordinates1, coordinates2):   # coordinates(type: list)- [long, lat]
    global client                            
    coordinates_set = [coordinates1, coordinates2]
    route_encoded = client.directions(coordinates_set, profile='driving-hgv', units='mi')['routes'][0]['geometry']
    route_decoded = convert.decode_polyline(route_encoded)
    return route_decoded


#_____EQUIDISTANCE/EQUITIME__________________________________________________________________
def find_halfway(coordinates1, coordinates2, unit):    # coordinates(type: list)- [long, lat]   
    global client                                      # unit(type: str)- 'distance' for equidistance, 'duration' for equitime
    route_info = [coordinates1, coordinates2]
    route_summary = client.directions(route_info, profile= 'driving-hgv')['routes'][0]['summary']
    route_steps = client.directions(route_info, profile= 'driving-hgv')['routes'][0]['segments'][0]['steps']
    route_unit_remaining = route_summary[unit]/2
    for step in route_steps:
        route_unit_remaining -= step[unit]
        if route_unit_remaining <= 0:
            segment_unit_total = step[unit]
            segment_waypoints = step['way_points'][1] - step['way_points'][0]
            waypoint_duration = segment_unit_total / segment_waypoints
            waypoints_remaining_units = round(route_unit_remaining / waypoint_duration)
            halfway_waypoint = waypoints_remaining_units + step['way_points'][1]
            return halfway_waypoint

#_____IPYWIDGETS: USER INPUT VALIDATION__________________________________________________________________        
def check_increment(r, i):
    if i == 0:
        return i
    while True:
        if r/i <= 10:
            return i
            break
        else:
            i += 1

In [114]:
#_____IPYWIDGETS: BRAGG__________________________________________________________________  
class Bragg:
    def __init__(self, name, latitude, longitude):
        self.name = name
        self.style = {'color':'#000000',  'weight': '1', 'fillColor': '#fcb615' , 'fillOpacity': '0.07'}
        self.latitude = latitude
        self.longitude = longitude
        self.coordinates = [latitude, longitude]
        self.measurement = widgets.Dropdown(
                    options = [('Miles (max 200)', 'distance'), ('Minutes (max 60)', 'time')],
                    tooltips = ['Limit: 200', 'Limit: 60'],
                    disabled = False,
                    continuous_update=False,
                    layout = other_lay)
        self.range = widgets.BoundedIntText(
                    value = 35,
                    min = 0,
                    max = 200,
                    step = 5,
                    disabled = False,
                    continuous_update=False,
                    layout = other_lay)
        self.increment = widgets.BoundedIntText(
                    value = 5,
                    min = 0,
                    max = 100,
                    step = 5,
                    disabled = False,
                    continuous_update=False,
                    layout = other_lay)
        self.smoothing = widgets.IntSlider(
                    value = 30,
                    min = 0,
                    max = 100,
                    step = 5,
                    disabled = False,
                    continuous_update = False,
                    orientation = 'horizontal',
                    readout = False,
                    layout = slider_lay)
        self.smoothing.style.handle_color = '#fcb615'
        self.add = widgets.Button(
                    description = "ADD/UPDATE", 
                    icon = 'fa-plus-square-o', 
                    layout = button_lay_add,
                    style = addb_style)
        self.remove = widgets.Button(
                    description = "REMOVE", 
                    icon = 'fa-minus-square-o', 
                    button_style = 'danger', 
                    layout = button_lay_remove,
                    style = removeb_style)
        self.geojson = leaf.GeoJSON(
                    style = self.style, 
                    name = self.name)
        self.icon = leaf.Icon(
                    icon_url = 'https://i.ibb.co/CzN4Bcv/logo.png', 
                    icon_size = [46,46], 
                    icon_anchor = [23,46])
        self.pin = leaf.Marker(
                    location = (self.latitude, self.longitude), 
                    title = self.name, 
                    icon = self.icon)
        self.folium_iso = None
        self.folium_pin = f.Marker(
                    [self.latitude, self.longitude], 
                    tooltip = self.name, 
                    icon = f.features.CustomIcon('https://i.ibb.co/CzN4Bcv/logo.png', icon_size=(50,50), icon_anchor=(25,50), popup_anchor=(0,-45)))
        self.hbox_measurement = widgets.HBox([widgets.Label(value='Measurement Unit'), self.measurement], layout=hbox_lay)
        self.hbox_range = widgets.HBox([widgets.Label(value='Total Range'), self.range], layout=hbox_lay)
        self.hbox_increment = widgets.HBox([widgets.Label(value='Increment By'), self.increment], layout=hbox_lay)
        self.hbox_smoothing = widgets.HBox([widgets.Label(value='Perimeter Granularity'), widgets.HBox(children=[widgets.Label(value='High'), self.smoothing, widgets.Label(value='Low')])], layout=hbox_lay)
        self.hbox_buttons = widgets.HBox([self.add, self.remove], layout=buttons_lay)
        self.vbox = widgets.VBox([self.hbox_measurement, self.hbox_range, self.hbox_increment, self.hbox_smoothing, self.hbox_buttons], layout=vbox_lay)
        def bragg_add(b):
            selected = accordion_bragg_items[accordion_bragg.selected_index]
            if (selected.range.value > 60 and selected.measurement.value == 'time'):
                selected.range.value = 60
            increment = check_increment(selected.range.value, selected.increment.value)
            selected.increment.value = increment
            iso = compute_poly(selected.latitude, selected.longitude, selected.measurement.value, selected.range.value, increment, selected.smoothing.value)
            selected.geojson.data = iso
            selected.folium_iso = f.GeoJson(iso, name=selected.name, style_function=style_bragg, zoom_on_click=False)
            for item in folium_layers_bragg:
                if item == selected:
                    folium_layers_bragg.pop(folium_layers_bragg.index(item))
            folium_layers_bragg.append(selected)
            try:
                bragg_map.add_layer(selected.geojson)
                bragg_map.add_layer(selected.pin)
            except:
                bragg_map.substitute_layer(selected.geojson, selected.geojson)
        def bragg_remove(b): 
            selected = accordion_bragg_items[accordion_bragg.selected_index]
            for item in folium_layers_bragg:
                if item == selected:
                    folium_layers_bragg.pop(folium_layers_bragg.index(item))
            try:
                bragg_map.remove_layer(selected.geojson)
                bragg_map.remove_layer(selected.pin)
            except:
                with out:
                    print("no object to remove")
        self.add.on_click(bragg_add)
        self.remove.on_click(bragg_remove)
    
    
longbeach = Bragg('Long Beach', 33.8689814, -118.1600415)
santamaria = Bragg('Santa Maria', 34.9238204, -120.4085693)
fontana = Bragg('Fontana',34.0525204, -117.5210737)
stockton = Bragg('Stockton', 37.9751619, -121.2625948)
mojave = Bragg('Mojave', 35.0533382, -118.1574785)
lakeside = Bragg('Lakeside', 32.909049, -116.947889)
sparks = Bragg('Sparks', 39.5215941, -119.6986499)
phoenix = Bragg('Phoenix', 33.4792082, -112.1171424)
woodscross = Bragg('Woods Cross', 40.8646155, -111.9416152)
odessa = Bragg('Odessa', 31.8437444, -102.3406495)
mtvernon = Bragg('Mt. Vernon', 48.431160, -122.369580)
meridian = Bragg('Meridian', 43.5903053, -116.3686752)
merritisland = Bragg('Merritt Island', 28.4411239, -80.70889282)
titusville = Bragg('Titusville', 28.5262963, -80.8178440)


accordion_bragg = widgets.Accordion(children=[longbeach.vbox, santamaria.vbox, fontana.vbox, stockton.vbox, mojave.vbox, lakeside.vbox, sparks.vbox, phoenix.vbox, woodscross.vbox, odessa.vbox, mtvernon.vbox, meridian.vbox, merritisland.vbox, titusville.vbox], selected_index=None)    
accordion_bragg_items = [longbeach, santamaria, fontana, stockton, mojave, lakeside, sparks, phoenix, woodscross, odessa, mtvernon, meridian, merritisland, titusville]
folium_layers_bragg = []


    
for item in range(len(accordion_bragg_items)):
    accordion_bragg.set_title(item, accordion_bragg_items[item].name)


In [115]:
#_____IPYWIDGETS: JOBSITE__________________________________________________________________  
class Jobsite:
    def __init__(self):
        self.style = { 'color': '#006600', 'weight': '1',  'fillColor': '#006600', 'fillOpacity': '0.07'}
        self.name = widgets.Text(
                    placeholder = 'Job Name',
                    disabled = False,
                    layout = other_lay) 
        self.latitude = widgets.BoundedFloatText(
                    min = -90,
                    max = 90,
                    step = 0.00000001,
                    disabled = False,
                    layout = coord_lay)
        self.longitude = widgets.BoundedFloatText(
                    min = -180,
                    max = 180,
                    step = 0.00000001,
                    disabled = False,
                    layout = coord_lay)
        self.measurement = widgets.Dropdown(
                    options = [('Miles (max 200)', 'distance'), ('Minutes (max 60)', 'time')],
                    tooltips = ['Limit: 200', 'Limit: 60'],
                    disabled = False,
                    continuous_update=False,
                    layout = other_lay)
        self.range = widgets.BoundedIntText(
                    value = 35,
                    min = 0,
                    max = 200,
                    step = 5,
                    disabled = False,
                    layout = other_lay)
        self.increment = widgets.BoundedIntText(
                    value = 5,
                    min = 0,
                    max = 100,
                    step = 5,
                    disabled = False,
                    layout = other_lay )
        self.smoothing = widgets.IntSlider(
                    value = 25,
                    min = 0,
                    max = 100,
                    step = 5,
                    disabled = False,
                    continuous_update = False,
                    orientation = 'horizontal',
                    readout = False,
                    layout = slider_lay)
        self.smoothing.style.handle_color = '#FFCC00'
        self.add = widgets.Button(
                    description = "ADD/UPDATE", 
                    icon = 'fa-plus-square-o', 
                    layout = button_lay_add,
                    style = addb_style)
        self.remove = widgets.Button(
                    description = "REMOVE", 
                    icon = 'fa-minus-square-o', 
                    button_style = 'danger', 
                    layout = button_lay_remove,
                    style = removeb_style)
        self.icon = leaf.Icon(
                    icon_url = 'https://i.ibb.co/9tr2F6B/hardhat.png', 
                    icon_size = [46,46], 
                    icon_anchor = [23,46])
        self.folium_iso = None
        self.folium_pin = None
        self.pin = leaf.Marker(icon = self.icon)
        self.geojson = leaf.GeoJSON(style = self.style)
        self.hbox_name = widgets.HBox([widgets.Label(value='Location ID'), self.name], layout=hbox_lay)
        self.hbox_coordinates = widgets.HBox([widgets.Label(value='Coordinates'), widgets.HBox([self.latitude, self.longitude, widgets.Label(value='(Lat, Long)')])], layout=hbox_lay)
        self.hbox_measurement = widgets.HBox([widgets.Label(value='Measurement Unit'), self.measurement], layout=hbox_lay)
        self.hbox_range = widgets.HBox([widgets.Label(value='Total Range'), self.range], layout=hbox_lay)
        self.hbox_increment = widgets.HBox([widgets.Label(value='Increment By'), self.increment], layout=hbox_lay)
        self.hbox_smoothing = widgets.HBox([widgets.Label(value='Perimeter Granularity'), widgets.HBox([widgets.Label(value='High'), self.smoothing, widgets.Label(value='Low')])], layout=hbox_lay)
        self.hbox_buttons = widgets.HBox([self.add, self.remove], layout=buttons_lay)
        self.vbox = widgets.VBox([self.hbox_name, self.hbox_coordinates, self.hbox_measurement, self.hbox_range, self.hbox_increment, self.hbox_smoothing, self.hbox_buttons], layout=vbox_lay)
        def jobsite_add(b):
            selected = accordion_jobsite_items[accordion_jobsite.selected_index]
            accordion_jobsite.set_title(accordion_jobsite.selected_index, selected.name.value)  
            if (selected.range.value > 60 and selected.measurement.value == 'time'):
                selected.range.value = 60
            increment = check_increment(selected.range.value, selected.increment.value) 
            selected.increment.value = increment
            iso = compute_poly(selected.latitude.value, selected.longitude.value, selected.measurement.value, selected.range.value, increment, selected.smoothing.value)
            selected.geojson.data = iso
            selected.geojson.name = selected.name.value
            selected.folium_iso = f.GeoJson(iso, name=selected.name.value, style_function=style_jobsite, zoom_on_click=False)
            selected.folium_pin = f.Marker([selected.latitude.value, selected.longitude.value], tooltip=selected.name.value, icon=f.features.CustomIcon('https://i.ibb.co/9tr2F6B/hardhat.png', icon_size=(50,50), icon_anchor=(25,50), popup_anchor=(0,-45)))
            selected.pin.location = (selected.latitude.value, selected.longitude.value)
            selected.pin.title = selected.name.value
            for item in folium_layers_jobsite:
                if item == selected:
                    folium_layers_jobsite.pop(folium_layers_jobsite.index(item))
            folium_layers_jobsite.append(selected)
            try:
                bragg_map.add_layer(selected.geojson)
                bragg_map.add_layer(selected.pin)
            except:
                bragg_map.substitute_layer(selected.geojson, selected.geojson)
        def jobsite_remove(b): 
            selected = accordion_jobsite_items[accordion_jobsite.selected_index]
            for item in folium_layers_jobsite:
                if item == selected:
                    folium_layers_jobsite.pop(folium_layers_jobsite.index(item))
            try:
                bragg_map.remove_layer(selected.geojson)
                bragg_map.remove_layer(selected.pin)
            except:
                with out:
                    print("jobsite already removed")
        self.add.on_click(jobsite_add)
        self.remove.on_click(jobsite_remove)
        

jobsites = ['Jobsite 1', 'Jobsite 2', 'Jobsite 3', 'Jobsite 4', 'Jobsite 5', 'Jobsite 6', 'Jobsite 7', 'Jobsite 8', 'Jobsite 9', 'Jobsite 10', 'Jobsite 11', 'Jobsite 12', 'Jobsite 13', 'Jobsite 14', 'Jobsite 15']  

jobsite1 = Jobsite()
jobsite2 = Jobsite()
jobsite3 = Jobsite()
jobsite4 = Jobsite()
jobsite5 = Jobsite()
jobsite6 = Jobsite()
jobsite7 = Jobsite()
jobsite8 = Jobsite()
jobsite9 = Jobsite()
jobsite10 = Jobsite()
jobsite11 = Jobsite()
jobsite12 = Jobsite()
jobsite13 = Jobsite()
jobsite14 = Jobsite()
jobsite15 = Jobsite()


accordion_jobsite = widgets.Accordion(children=[jobsite1.vbox, jobsite2.vbox, jobsite3.vbox, jobsite4.vbox, jobsite5.vbox, jobsite6.vbox, jobsite7.vbox, jobsite8.vbox, jobsite9.vbox, jobsite10.vbox, jobsite11.vbox, jobsite12.vbox, jobsite13.vbox, jobsite14.vbox, jobsite15.vbox], selected_index = None)
accordion_jobsite_items = [jobsite1, jobsite2, jobsite3, jobsite4, jobsite5, jobsite6, jobsite7, jobsite8, jobsite9, jobsite10, jobsite11, jobsite12, jobsite13, jobsite14, jobsite15]
folium_layers_jobsite = []


for i in range(len(jobsites)):
    accordion_jobsite.set_title(i, jobsites[i])

    

        

In [116]:
#_____IPYWIDGETS: COMPETITOR__________________________________________________________________  
class Competitor:
    def __init__(self):
        self.style = {'color': '#CC0000', 'weight': '1',  'fillColor': '#CC0000', 'fillOpacity': '0.07'}
        self.name = widgets.Text(
                    placeholder = 'Competitor Name',
                    disabled = False,
                    layout = other_lay )
        self.latitude = widgets.BoundedFloatText(
                    min = -90,
                    max = 90,
                    step = 0.00000001,
                    disabled = False,
                    layout = coord_lay)
        self.longitude = widgets.BoundedFloatText(
                    min = -180,
                    max = 180,
                    step = 0.00000001,
                    disabled = False,
                    layout = coord_lay)
        self.measurement = widgets.Dropdown(
                    options = [('Miles (max 200)', 'distance'), ('Minutes (max 60)', 'time')],
                    tooltips = ['Limit: 200', 'Limit: 60'],
                    disabled = False,
                    continuous_update=False,
                    layout = other_lay)
        self.range = widgets.BoundedIntText(
                    value = 35,
                    min = 0,
                    max = 200,
                    step = 5,
                    disabled = False,
                    layout = other_lay )
        self.increment = widgets.BoundedIntText(
                    value = 5,
                    min = 0,
                    max = 100,
                    step = 5,
                    disabled = False,
                    layout = other_lay )
        self.smoothing = widgets.IntSlider(
                    value = 25,
                    min = 0,
                    max = 100,
                    step = 5,
                    disabled = False,
                    continuous_update = False,
                    orientation = 'horizontal',
                    readout = False,
                    layout = slider_lay)
        self.smoothing.style.handle_color = '#FFCC00'
        self.add = widgets.Button(
                    description = "ADD/UPDATE", 
                    icon = 'fa-plus-square-o', 
                    layout = button_lay_add,
                    style = addb_style)
        self.remove = widgets.Button(
                    description = "REMOVE", 
                    icon = 'fa-minus-square-o', 
                    button_style = 'danger', 
                    layout = button_lay_remove,
                    style = removeb_style)
        self.icon = leaf.Icon(
                    icon_url = 'https://i.ibb.co/2Mdfh6S/crane.png', 
                    icon_size = [46,46], 
                    icon_anchor = [23,46])
        self.folium_iso = None
        self.folium_pin = None
        self.pin = leaf.Marker(icon = self.icon)
        self.geojson = leaf.GeoJSON(style = self.style)
        self.hbox_name = widgets.HBox([widgets.Label(value='Location ID'), self.name], layout=hbox_lay)
        self.hbox_coordinates = widgets.HBox([widgets.Label(value='Coordinates'), widgets.HBox([self.latitude, self.longitude, widgets.Label(value='(Lat, Long)')])], layout=hbox_lay)
        self.hbox_measurement = widgets.HBox([widgets.Label(value='Measurement Unit'), self.measurement], layout=hbox_lay)
        self.hbox_range = widgets.HBox([widgets.Label(value='Total Range'), self.range], layout=hbox_lay)
        self.hbox_increment = widgets.HBox([widgets.Label(value='Increment By'), self.increment], layout=hbox_lay)
        self.hbox_smoothing = widgets.HBox([widgets.Label(value='Perimeter Granularity'), widgets.HBox([widgets.Label(value='High'), self.smoothing, widgets.Label(value='Low')])], layout=hbox_lay)
        self.hbox_buttons = widgets.HBox([self.add, self.remove], layout=buttons_lay)
        self.vbox = widgets.VBox([self.hbox_name, self.hbox_coordinates, self.hbox_measurement, self.hbox_range, self.hbox_increment, self.hbox_smoothing, self.hbox_buttons], layout=vbox_lay)
        def competitor_add(b):
            selected = accordion_competitor_items[accordion_competitor.selected_index]
            accordion_competitor.set_title(accordion_competitor.selected_index, selected.name.value)  
            if (selected.range.value > 60 and selected.measurement.value == 'time'):
                selected.range.value = 60
            increment = check_increment(selected.range.value, selected.increment.value) 
            selected.increment.value = increment
            iso = compute_poly(selected.latitude.value, selected.longitude.value, selected.measurement.value, selected.range.value, increment, selected.smoothing.value)
            selected.geojson.data = iso
            selected.geojson.name = selected.name.value
            selected.folium_iso = f.GeoJson(iso, name=selected.name.value, style_function=style_competitor, zoom_on_click=False)
            selected.folium_pin = f.Marker([selected.latitude.value, selected.longitude.value], tooltip=selected.name.value, icon=f.features.CustomIcon('https://i.ibb.co/2Mdfh6S/crane.png', icon_size=(50,50), icon_anchor=(25,50), popup_anchor=(0,-45)))
            selected.pin.location = (selected.latitude.value, selected.longitude.value)
            selected.pin.title = selected.name.value
            for item in folium_layers_competitor:
                if item == selected:
                    folium_layers_competitor.pop(folium_layers_competitor.index(item))
            folium_layers_competitor.append(selected)
            try:
                bragg_map.add_layer(selected.geojson)
                bragg_map.add_layer(selected.pin)
            except:
                bragg_map.substitute_layer(selected.geojson, selected.geojson)
        def competitor_remove(b): 
            selected = accordion_competitor_items[accordion_competitor.selected_index]
            for item in folium_layers_competitor:
                if item == selected:
                    folium_layers_competitor.pop(folium_layers_competitor.index(item))
            try:
                bragg_map.remove_layer(selected.geojson)
                bragg_map.remove_layer(selected.pin)
            except:
                with out:
                    print("competitor already removed")
        self.add.on_click(competitor_add)
        self.remove.on_click(competitor_remove)

competitors = ['Competitor 1', 'Competitor 2', 'Competitor 3', 'Competitor 4', 'Competitor 5', 'Competitor 6', 'Competitor 7', 'Competitor 8', 'Competitor 9', 'Competitor 10', 'Competitor 11', 'Competitor 12', 'Competitor 13', 'Competitor 14', 'Competitor 15']  

competitor1 = Competitor()
competitor2 = Competitor()
competitor3 = Competitor()
competitor4 = Competitor()
competitor5 = Competitor()
competitor6 = Competitor()
competitor7 = Competitor()
competitor8 = Competitor()
competitor9 = Competitor()
competitor10 = Competitor()
competitor11 = Competitor()
competitor12 = Competitor()
competitor13 = Competitor()
competitor14 = Competitor()
competitor15 = Competitor()

accordion_competitor = widgets.Accordion(children=[competitor1.vbox, competitor2.vbox, competitor3.vbox, competitor4.vbox, competitor5.vbox, competitor6.vbox, competitor7.vbox, competitor8.vbox, competitor9.vbox, competitor10.vbox, competitor11.vbox, competitor12.vbox, competitor13.vbox, competitor14.vbox, competitor15.vbox], selected_index = None)
accordion_competitor_items = [competitor1, competitor2, competitor3, competitor4, competitor5, competitor6, competitor7, competitor8, competitor9, competitor10, competitor11, competitor12, competitor13, competitor14, competitor15]
folium_layers_competitor = []



for i in range(len(competitors)):
    accordion_competitor.set_title(i, competitors[i])
    


In [117]:
#_____IPYWIDGETS: ROUTE__________________________________________________________________  
class Route:
    def __init__(self):
        self.style = {'color':'#FD1C03', 'weight':'2'}
        self.name = widgets.Text(
                    placeholder = 'Route Name',
                    disabled = False,
                    layout = other_lay)
        self.latitude1 = widgets.BoundedFloatText(
                    min = -90,
                    max = 90,
                    step = 0.00000001,
                    disabled = False,
                    layout = coord_lay)
        self.longitude1 = widgets.BoundedFloatText(
                    min = -180,
                    max = 180,
                    step = 0.00000001,
                    disabled = False,
                    layout = coord_lay)
        self.latitude2 = widgets.BoundedFloatText(
                    min = -90,
                    max = 90,
                    step = 0.00000001,
                    disabled = False,
                    layout = coord_lay)
        self.longitude2 = widgets.BoundedFloatText(
                    min = -180,
                    max = 180,
                    step = 0.00000001,
                    disabled = False,
                    layout = coord_lay)
        self.type = widgets.Dropdown(
                    options = ['Equitime', 'Equidistant', 'None'],
                    disabled = False,
                    orientation = 'horizontal',
                    layout = other_lay)
        self.add = widgets.Button(
                    description = "ADD/UPDATE", 
                    icon = 'fa-plus-square-o', 
                    layout = button_lay_add,
                    style = addb_style)
        self.remove = widgets.Button(
                    description = "REMOVE", 
                    icon = 'fa-minus-square-o', 
                    button_style = 'danger', 
                    layout = button_lay_remove,
                    style = removeb_style)
        self.equidistant_icon = AwesomeIcon(
                    name='road', 
                    marker_color='darkred', 
                    icon_color='beige',
                    icon_size = [46,46], 
                    icon_anchor = [23,46])
        self.equitime_icon = AwesomeIcon(
                    name='hourglass', 
                    marker_color='darkgreen', 
                    icon_color='beige',
                    icon_size = [46,46], 
                    icon_anchor = [23,46])
        self.folium_route = None
        self.folium_pin = None
        self.geojson = leaf.GeoJSON(style=self.style)
        self.pin = leaf.Marker()
        self.hbox_name = widgets.HBox([widgets.Label(value='Route ID'), self.name], layout=hbox_lay)
        self.hbox_coordinates1 = widgets.HBox([widgets.Label(value='Coordinate Set 1'), widgets.HBox([self.latitude1, self.longitude1, widgets.Label(value='(Lat, Long)')])], layout=hbox_lay)
        self.hbox_coordinates2 = widgets.HBox([widgets.Label(value='Coordinate Set 2'), widgets.HBox([self.latitude2, self.longitude2, widgets.Label(value='(Lat, Long)')])], layout=hbox_lay)
        self.hbox_type = widgets.HBox([widgets.Label(value='Display'), self.type], layout=hbox_lay)
        self.hbox_buttons = widgets.HBox([self.add, self.remove], layout=buttons_lay)
        self.vbox = widgets.VBox([self.hbox_name, self.hbox_coordinates1, self.hbox_coordinates2, self.hbox_type, self.hbox_buttons], layout=vbox_lay)
        def route_add(b):
            selected = accordion_route_items[accordion_route.selected_index]
            accordion_route.set_title(accordion_route.selected_index, selected.name.value)
            coordinates1 = [selected.longitude1.value, selected.latitude1.value]
            coordinates2 = [selected.longitude2.value, selected.latitude2.value]
            route = find_route(coordinates1, coordinates2)
            selected.geojson.data = route
            selected.geojson.name = selected.name.value
            selected.folium_route = f.GeoJson(route, name=selected.name.value, tooltip=selected.name.value, style_function=style_route, zoom_on_click=False)
            try:
                bragg_map.add_layer(selected.geojson)
            except:
                bragg_map.substitute_layer(selected.geojson, selected.geojson)
            if selected.type.value == 'Equitime':
                pin_coord = route['coordinates'][find_halfway(coordinates1, coordinates2, 'duration')]
                name_equitime = selected.name.value + ': Equitime'
                selected.folium_pin = f.Marker([pin_coord[1], pin_coord[0]], tooltip=name_equitime, icon=f.Icon(icon='hourglass', color='darkgreen', icon_color='beige', prefix = 'fa'))
                selected.pin.location = (pin_coord[1], pin_coord[0])
                selected.pin.icon = selected.equitime_icon
                selected.pin.title = selected.name.value + ': Equitime'
                try:
                    bragg_map.add_layer(selected.pin)
                except:
                    bragg_map.substitute_layer(selected.pin, selected.pin)
            elif selected.type.value == 'Equidistant':
                pin_coord = route['coordinates'][find_halfway(coordinates1, coordinates2, 'distance')]
                name_equidistant = selected.name.value + ': Equidistant'
                selected.folium_pin = f.Marker([pin_coord[1], pin_coord[0]], tooltip=name_equidistant, icon=f.Icon(icon='road', color='darkred', icon_color='beige', prefix = 'fa'))
                selected.pin.location = (pin_coord[1], pin_coord[0])
                selected.pin.icon = selected.equidistant_icon
                selected.pin.title = selected.name.value + ': Equidistant'
                try:
                    bragg_map.add_layer(selected.pin)
                except:
                    bragg_map.substitute_layer(selected.pin, selected.pin)
            else:
                try:
                    bragg_map.remove_layer(selected.pin)
                except:
                    pass
            for item in folium_layers_route:
                if item == selected:
                    folium_layers_route.pop(folium_layers_route.index(item))
            folium_layers_route.append(selected)
        def route_remove(b): 
            selected = accordion_route_items[accordion_route.selected_index]
            try:
                bragg_map.remove_layer(selected.geojson)
                bragg_map.remove_layer(selected.pin)
            except:
                with out:
                    print("route already removed")
            for item in folium_layers_route:
                if item == selected:
                    folium_layers_route.pop(folium_layers_route.index(item))
        self.add.on_click(route_add)
        self.remove.on_click(route_remove)
        
        
routes = ['Route 1', 'Route 2', 'Route 3', 'Route 4', 'Route 5', 'Route 6', 'Route 7', 'Route 8', 'Route 9', 'Route 10', 'Route 11', 'Route 12', 'Route 13', 'Route 14', 'Route 15']  

route1 = Route()
route2 = Route()
route3 = Route()
route4 = Route()
route5 = Route()
route6 = Route()
route7 = Route()
route8 = Route()
route9 = Route()
route10 = Route()
route11 = Route()
route12 = Route()
route13 = Route()
route14 = Route()
route15 = Route()

accordion_route = widgets.Accordion(children=[route1.vbox, route2.vbox, route3.vbox, route4.vbox, route5.vbox, route6.vbox, route7.vbox, route8.vbox, route9.vbox, route10.vbox, route11.vbox, route12.vbox, route13.vbox, route14.vbox, route15.vbox], selected_index=None)
accordion_route_items = [route1, route2, route3, route4, route5, route6, route7, route8, route9, route10, route11, route12, route13, route14, route15]
folium_layers_route = []

for i in range(len(routes)):
    accordion_route.set_title(i, routes[i])

    

    

In [118]:
folium_map = f.Map(location=center, tiles='OpenStreetMap',  zoom_start=5, zoom_control=True, min_zoom=3, max_zoom=18)
filename_save = widgets.Text(placeholder = 'Filename', disabled = False, layout=widgets.Layout(height='100%'))
button_save = widgets.Button(style=widgets.ButtonStyle(button_color='#B0C9F6', font_weight='bold'), icon='floppy-o', layout=widgets.Layout(width='7.5%', height='65%', padding='.5px .5px .5px .5px'))
hbox_save = widgets.HBox([filename_save, button_save])
message_save = widgets.Label(value="")
vbox_save = widgets.VBox([hbox_save, message_save])
def save_map(b):
    for item in folium_layers_bragg:
        item.folium_iso.add_to(folium_map)
        item.folium_pin.add_to(folium_map)
    for item in folium_layers_jobsite:
        item.folium_iso.add_to(folium_map)
        item.folium_pin.add_to(folium_map)
    for item in folium_layers_competitor:
        item.folium_iso.add_to(folium_map)
        item.folium_pin.add_to(folium_map)
    for item in folium_layers_route:
        item.folium_route.add_to(folium_map)
        item.folium_pin.add_to(folium_map)
    message_save.value = 'map saved!'
    folium_map.save(filename_save.value + '.html')
button_save.on_click(save_map)

In [119]:
tabs = widgets.Tab([accordion_bragg, accordion_jobsite, accordion_competitor, accordion_route, vbox_save], layout=widgets.Layout(width='535px', height='auto'))
tabs.set_title(0, 'BRAGG')
tabs.set_title(1, 'JOBSITE')
tabs.set_title(2, 'COMPETITOR')
tabs.set_title(3, 'ROUTE')
tabs.set_title(4, 'SAVE')




In [120]:
out = widgets.Output()
app_grid = widgets.GridspecLayout(1, 3)
app_grid[:, 2] = tabs
app_grid[:, :2] = bragg_map
app = widgets.VBox([app_grid, out], layout=widgets.Layout(margin='0px 1px auto 1px', grid_gap='0px'))

app
    


